In [1]:
using DataStructures
using CPUTime
using DelimitedFiles
using Base
using Distributed

In [2]:
numVertex=4
Edges = [(4,1,2), (1,2,1), (2,3,1), (3,4,1), (4,2,3), (1,3,5)]

6-element Array{Tuple{Int64,Int64,Int64},1}:
 (4, 1, 2)
 (1, 2, 1)
 (2, 3, 1)
 (3, 4, 1)
 (4, 2, 3)
 (1, 3, 5)

In [3]:
numVertex=6
Edges = [
(6, 1, 1),
(6, 2, 3),
(1, 2, 1),
(1, 3, 7),
(2, 4, 1),
(1, 4, 3),
(3, 4, 1),
(3, 5, 1),
(4, 5, 6)]
# source, destionation, weight

9-element Array{Tuple{Int64,Int64,Int64},1}:
 (6, 1, 1)
 (6, 2, 3)
 (1, 2, 1)
 (1, 3, 7)
 (2, 4, 1)
 (1, 4, 3)
 (3, 4, 1)
 (3, 5, 1)
 (4, 5, 6)

In [2]:
Edges = Vector{}()
function readDataFromFile(filename="in_n_10e5_m_5e6.in")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
        sizehint!(Edges, parse(Int, m))
#         println(n," ",m)
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
            push!(Edges, (a,b,c))
#             println(a," ",b," ",c)
        end
        return n
    end
end
numVertex = readDataFromFile()

100000

In [38]:
uf = IntDisjointSets(numVertex+1)

weights = Vector{}()
sizehint!(weights, length(Edges))

mst= Vector{}()
sizehint!(mst, numVertex)

0-element Array{Any,1}

In [39]:
for e in Edges
    push!(weights, (e[3], (src=e[1],dst= e[2])))
end

In [26]:
Base.@pure function kruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    sortedEdgeWeights = sort(EdgeWeights)
    mst_weight=0
    for weight in sortedEdgeWeights
        c, e = weight
        if !in_same_set(uf, e.src, e.dst)
            union!(uf, e.src, e.dst)
            push!(mst, e)    
            mst_weight+=c
        end
    end
    return mst_weight
end    

kruskal (generic function with 1 method)

In [27]:
using Random
rng = MersenneTwister();

In [30]:
Base.@pure function filterKruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    if kruskalThreshold(length(EdgeWeights), length(MST))
        # println("pre-start kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        tmp =  @async kruskal(EdgeWeights, MST, UF)
        # println("allocated kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        res = fetch(tmp)
        # println("finished kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        return res
    else
        sampler = Random.Sampler(rng, EdgeWeights)
        pivot = rand(rng, sampler)
        
        # Partition
        EdgeWeightsLeft = Vector{}()
        sizehint!(EdgeWeightsLeft, length(EdgeWeights)÷2)

        EdgeWeightsRight = Vector{}()
        sizehint!(EdgeWeightsRight, length(EdgeWeights)÷2)

        for EdgeWeight in EdgeWeights
            if EdgeWeight[1] < pivot[1]
                push!(EdgeWeightsLeft, EdgeWeight)
            else
                push!(EdgeWeightsRight, EdgeWeight)
            end
        end
        
#         pmap(x->(x[1]<pivot[1]) ? push!(EdgeWeightsLeft, x) : push!(EdgeWeightsRight, x), EdgeWeights)
#         println("pivot = ",pivot)
#         println("left = ",length(EdgeWeightsLeft))
#         println("right = ",length(EdgeWeightsRight))
#         println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," going to start at ", CPUtime_us())
        leftMST = @async filterKruskal(EdgeWeightsLeft, MST, UF)
        FilteredEdgeWeightsRight = filter(EdgeWeightsRight, UF)
        rightMST = @async filterKruskal(FilteredEdgeWeightsRight, MST, UF)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," allocated at ", CPUtime_us())
        res = fetch(leftMST) + fetch(rightMST)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," finished at ", CPUtime_us())
        return res
    end
end

filterKruskal (generic function with 1 method)

In [32]:
Base.@pure function filterKruskal(EdgeWeights, MST, UF)
    if isempty(EdgeWeights)
        return 0
    end
    if kruskalThreshold(length(EdgeWeights), length(MST))
        # println("pre-start kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        tmp = kruskal(EdgeWeights, MST, UF)
        # println("allocated kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        res = tmp
        # println("finished kruskal ",length(EdgeWeights) ," at", CPUtime_us())
        return res
    else
        sampler = Random.Sampler(rng, EdgeWeights)
        pivot = rand(rng, sampler)
        
        # Partition
        EdgeWeightsLeft = Vector{}()
        sizehint!(EdgeWeightsLeft, length(EdgeWeights)÷2)

        EdgeWeightsRight = Vector{}()
        sizehint!(EdgeWeightsRight, length(EdgeWeights)÷2)

        for EdgeWeight in EdgeWeights
            if EdgeWeight[1] < pivot[1]
                push!(EdgeWeightsLeft, EdgeWeight)
            else
                push!(EdgeWeightsRight, EdgeWeight)
            end
        end
        
#         pmap(x->(x[1]<pivot[1]) ? push!(EdgeWeightsLeft, x) : push!(EdgeWeightsRight, x), EdgeWeights)
#         println("pivot = ",pivot)
#         println("left = ",length(EdgeWeightsLeft))
#         println("right = ",length(EdgeWeightsRight))
#         println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," going to start at ", CPUtime_us())
        leftMST = filterKruskal(EdgeWeightsLeft, MST, UF)
        FilteredEdgeWeightsRight = filter(EdgeWeightsRight, UF)
        rightMST = filterKruskal(FilteredEdgeWeightsRight, MST, UF)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," allocated at ", CPUtime_us())
        res = (leftMST) + (rightMST)
        # println("Partition ",length(EdgeWeightsLeft)," ",length(EdgeWeightsRight) ," finished at ", CPUtime_us())
        return res
    end
end

filterKruskal (generic function with 1 method)

In [33]:
Base.@pure function kruskalThreshold(EdgesLength, MSTLength)
    sp = Random.Sampler(rng, 1:EdgesLength) # or Random.Sampler(MersenneTwister, 1:20)
#     return EdgesLength <=10000
    return EdgesLength/1.25<=rand(rng, sp) || EdgesLength <=1000
end

kruskalThreshold (generic function with 1 method)

In [34]:
Base.@pure function filter(EdgeWeights, UF)
    FilteredEdgeWeights = Vector{}()
    for EdgeWeight in EdgeWeights
        if !in_same_set(UF, EdgeWeight[2].src, EdgeWeight[2].dst)
            push!(FilteredEdgeWeights, EdgeWeight)
        end
    end
    return FilteredEdgeWeights
end

filter (generic function with 1 method)

In [40]:
@time @CPUtime filterKruskal(weights, mst, uf)

elapsed CPU time: 14.057086 seconds
 14.579658 seconds (70.69 M allocations: 1.708 GiB, 12.42% gc time)


5996149158

In [21]:
@time @CPUtime kruskal(weights, mst, uf)


elapsed CPU time: 13.178604 seconds
 13.544814 seconds (70.69 M allocations: 1.708 GiB, 12.08% gc time)


5996149158

In [ ]:
Threads.nthreads()